### Fetching the data

In [2]:
# 01_data_collection_m1.ipynb

import MetaTrader5 as mt5
import pandas as pd
from datetime import datetime, timedelta, timezone
import os

# --- CONFIG ---
SYMBOL = "EURUSD"
TIMEFRAME = mt5.TIMEFRAME_M1  # <--- CRITICAL: Fetching M1, not H1
START_YEAR = 2020
END_YEAR = 2026
DATA_DIR = "../data"

# --- INIT ---
# Tries to connect to your MT5 terminal
if not mt5.initialize():
    print(f"❌ Initialize failed: {mt5.last_error()}")
    quit()

os.makedirs(DATA_DIR, exist_ok=True)

# --- CHUNK FETCHING FUNCTION ---
def fetch_data_in_chunks(symbol, timeframe, start_year, end_year):
    all_dfs = []
    
    # Iterate through years to avoid MT5 timeout
    for year in range(start_year, end_year + 1):
        
        # Start (Jan 1) and end (Dec 31) of the current year in the loop
        t_start = datetime(year, 1, 1, tzinfo=timezone.utc)
        t_end = datetime(year, 12, 31, 23, 59, tzinfo=timezone.utc)
        
        # Don't fetch future
        if t_start > datetime.now(timezone.utc):
            break
        if t_end > datetime.now(timezone.utc):
            t_end = datetime.now(timezone.utc)

        print(f"⏳ Fetching {year}...")
        rates = mt5.copy_rates_range(symbol, timeframe, t_start, t_end)
        
        if rates is not None and len(rates) > 0:
            df_chunk = pd.DataFrame(rates)
            df_chunk['time'] = pd.to_datetime(df_chunk['time'], unit='s')
            all_dfs.append(df_chunk)
        else:
            print(f"⚠️ No data for {year}")

    if not all_dfs:
        return None
        
    # Combine all chunks
    final_df = pd.concat(all_dfs).drop_duplicates(subset='time').sort_values('time')
    final_df.set_index('time', inplace=True)
    return final_df[['open', 'high', 'low', 'close', 'tick_volume']]

# --- EXECUTION ---
print(f"🚀 Starting M1 Extraction for {SYMBOL}...")
df_m1 = fetch_data_in_chunks(SYMBOL, TIMEFRAME, START_YEAR, END_YEAR)

if df_m1 is not None:
    # Save as Parquet (Much faster/smaller than CSV for M1 data)
    save_path = os.path.join(DATA_DIR, f"{SYMBOL}_M1_Raw.parquet")
    df_m1.to_parquet(save_path)
    print(f"✅ Success! Saved {len(df_m1)} rows to {save_path}")
    print(df_m1.tail())
else:
    print("❌ Failed to fetch data.")

mt5.shutdown()

🚀 Starting M1 Extraction for EURUSD...
⏳ Fetching 2020...
⏳ Fetching 2021...
⏳ Fetching 2022...
⏳ Fetching 2023...
⏳ Fetching 2024...
⏳ Fetching 2025...
⏳ Fetching 2026...
⚠️ No data for 2026
✅ Success! Saved 2223168 rows to ../data\EURUSD_M1_Raw.parquet
                        open     high      low    close  tick_volume
time                                                                
2025-12-31 23:54:00  1.17459  1.17460  1.17457  1.17458            7
2025-12-31 23:55:00  1.17463  1.17463  1.17453  1.17455           18
2025-12-31 23:56:00  1.17455  1.17455  1.17454  1.17454            2
2025-12-31 23:57:00  1.17454  1.17455  1.17453  1.17454           11
2025-12-31 23:58:00  1.17453  1.17453  1.17453  1.17453            1


True